# Common utitities for all production alphas

In [ ]:
function get_recommendee_username()
    dir = "../../data/recommendations"
    open("$dir/recommendee_username.txt") do f
        read(f, String)
    end    
end;

In [ ]:
name = "../../data/recommendations/$(get_recommendee_username())/alphas/$source"
residual_alphas = [];

In [ ]:
@nbinclude("../TrainingAlphas/Alpha.ipynb");

In [ ]:
using DataFrames

In [ ]:
function get_recommendee_list()
    username = get_recommendee_username()
    dir = "../../data/recommendations"
    file = "$dir/$username/user_anime_list.jld2"
    load(file)["user_anime_list"]
end;

In [ ]:
function num_items()
    df = DataFrame(CSV.File("../../data/processed_data/anime_to_uid.csv"))
    length(df.uid)
end

In [ ]:
function write_recommendee_alpha(model; outdir=source)
    items = 1:num_items()
    training = get_recommendee_list()
    users = fill(training.user[1], length(items))
    alpha = RatingsDataset(users, items, model(items))

    dir = "../../data/recommendations"
    username = get_recommendee_username()
    outdir = mkpath("$dir/$username/alphas/$outdir")
    save("$outdir/alpha.jld2", Dict("alpha" => alpha))
end;

In [ ]:
function read_recommendee_alpha(alpha)
    dir = "../../data/recommendations"
    username = get_recommendee_username()
    load("$dir/$username/alphas/$alpha/alpha.jld2")["alpha"]
end;

In [ ]:
function get_recommendee_residuals(alphas)
    # train a linear model on the validation set
    y = get_split("validation").rating
    X = zeros(length(y), length(alphas))
    for j = 1:length(alphas)
        X[:, j] = get_alpha(alphas[j], "validation").rating
    end
    β = X \ y

    # residualize out the linear model
    df = get_recommendee_list()
    y = df.rating
    X = zeros(length(y), length(alphas))
    for j = 1:length(alphas)
        X[:, j] = read_recommendee_alpha(alphas[j]).rating[df.item]
    end
    ratings = y - X * β
    RatingsDataset(df.user, df.item, ratings)
end